## InfoSec Jupyterthon 2021 - Day 2

---
# 3. Advanced Pandas

- Joins and merges [Ian, Ashwin]
- Using styles [Ian, Pete] 
- Reshaping/preprocessing data [Ashwin, Luis Francisco, Ian] 
- Time manipulation [Ashwin]
- Other useful operations  [Luis Francisco] 


---

# Joins and merges [Ian, Ashwin] 

**Pandas UserGuide :** [Merge, join, concatenate and compare](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)


 ### Load some data and normalize it into:
 - Processes
 - ParentProcesses
 - Users

In [58]:
import pandas as pd

procs_df = pd.read_csv(
    "../data/process_tree.csv",
    parse_dates=["TimeCreatedUtc", "TimeGenerated"],
    index_col=0
)
parents = procs_df[["ProcessId", "ParentProcessName"]].drop_duplicates()
procs = (
    procs_df[["NewProcessId", "NewProcessName", "CommandLine", "ProcessId", "TimeCreatedUtc", "SubjectUserSid"]]
    .drop_duplicates()
    .rename(columns={"ProcessId": "ParentProcessId"})
)
users = procs_df[['SubjectUserSid', 'SubjectUserName', 'SubjectDomainName']].drop_duplicates()

print("original", len(procs_df))
print("procs", len(procs))
print("parents", len(parents))
print("users", len(users))
display(procs.head(3))
display(parents)
display(users)

original 117
procs 117
parents 3
users 2


NewProcessId                  NewProcessName  \
0       0x1580  C:\Diagnostics\UserTmp\ftp.exe   
1       0x16fc  C:\Diagnostics\UserTmp\reg.exe   
2       0x1700  C:\Diagnostics\UserTmp\cmd.exe   

                                         CommandLine ParentProcessId  \
0                   .\ftp  -s:C:\RECYCLER\xxppyy.exe           0xbc8   
1  .\reg  not /domain:everything that /sid:shines...           0xbc8   
2                 cmd  /c "systeminfo && systeminfo"           0xbc8   

           TimeCreatedUtc                                SubjectUserSid  
0 2019-01-15 05:15:15.677  S-1-5-21-996632719-2361334927-4038480536-500  
1 2019-01-15 05:15:16.167  S-1-5-21-996632719-2361334927-4038480536-500  
2 2019-01-15 05:15:16.277  S-1-5-21-996632719-2361334927-4038480536-500

ProcessId                      ParentProcessName
0       0xbc8            C:\Windows\System32\cmd.exe
115     0x440        C:\Windows\System32\svchost.exe
116    0x1580  C:\Diagnostics\UserTmp\powershell.exe

SubjectUserSid   SubjectUserName  \
0    S-1-5-21-996632719-2361334927-4038480536-500        MSTICAdmin   
115                                      S-1-5-18  MSTICAlertsWin1$   

    SubjectDomainName  
0     MSTICAlertsWin1  
115         WORKGROUP

## Joining on Index using pd.concat

<p style="font-family:consolas; font-size:15pt; color:green">
pd.concat([df1, df2...])
</p>

We saw using pd.concat to append rows in part 1

In [2]:
# Do some processing on the original DF
dec_logon_id = (
    pd.DataFrame(procs_df.SubjectLogonId.apply(lambda x: int(x, base=16)))
    .rename(columns={"SubjectLogonId": "SubjectLogonId_dec"})
)

dec_logon_id.head(5)

SubjectLogonId_dec
0            16428071
1            16428071
2            16428071
3            16428071
4            16428071

#### pd.concat with `axis="columns"` or `axis=1` joins column-wise (horizontally)

In [3]:
(
    pd.concat([procs_df, dec_logon_id], axis="columns")
    .head()
    .filter(regex=".*Process.*|Sub.*")
)

SubjectUserSid SubjectUserName  \
0  S-1-5-21-996632719-2361334927-4038480536-500      MSTICAdmin   
1  S-1-5-21-996632719-2361334927-4038480536-500      MSTICAdmin   
2  S-1-5-21-996632719-2361334927-4038480536-500      MSTICAdmin   
3  S-1-5-21-996632719-2361334927-4038480536-500      MSTICAdmin   
4  S-1-5-21-996632719-2361334927-4038480536-500      MSTICAdmin   

  SubjectDomainName SubjectLogonId NewProcessId  \
0   MSTICAlertsWin1       0xfaac27       0x1580   
1   MSTICAlertsWin1       0xfaac27       0x16fc   
2   MSTICAlertsWin1       0xfaac27       0x1700   
3   MSTICAlertsWin1       0xfaac27       0x1728   
4   MSTICAlertsWin1       0xfaac27       0x175c   

                        NewProcessName ProcessId            ParentProcessName  \
0       C:\Diagnostics\UserTmp\ftp.exe     0xbc8  C:\Windows\System32\cmd.exe   
1       C:\Diagnostics\UserTmp\reg.exe     0xbc8  C:\Windows\System32\cmd.exe   
2       C:\Diagnostics\UserTmp\cmd.exe     0xbc8  C:\Windows\System32\cmd.exe   
3  C:\Diagnostics\UserTmp\rundll32.exe     0xbc8  C:\Windows\System32\cmd.exe   
4  C:\Diagnostics\UserTmp\rundll32.exe     0xbc8  C:\Windows\System32\cmd.exe   

   SubjectLogonId_dec  
0            16428071  
1            16428071  
2            16428071  
3            16428071  
4            16428071

## Key-based Joins

<p style="font-family:consolas; font-size:15pt; color:green">
df1.merge(df2, ...)<br>
df1.join(df2, ...)
</p>


Source tables

In [4]:
display(procs.head())
display(users)


NewProcessId                       NewProcessName  \
0       0x1580       C:\Diagnostics\UserTmp\ftp.exe   
1       0x16fc       C:\Diagnostics\UserTmp\reg.exe   
2       0x1700       C:\Diagnostics\UserTmp\cmd.exe   
3       0x1728  C:\Diagnostics\UserTmp\rundll32.exe   
4       0x175c  C:\Diagnostics\UserTmp\rundll32.exe   

                                         CommandLine ParentProcessId  \
0                   .\ftp  -s:C:\RECYCLER\xxppyy.exe           0xbc8   
1  .\reg  not /domain:everything that /sid:shines...           0xbc8   
2                 cmd  /c "systeminfo && systeminfo"           0xbc8   
3                           .\rundll32  /C 12345.exe           0xbc8   
4       .\rundll32  /C c:\users\MSTICAdmin\12345.exe           0xbc8   

           TimeCreatedUtc                                SubjectUserSid  
0 2019-01-15 05:15:15.677  S-1-5-21-996632719-2361334927-4038480536-500  
1 2019-01-15 05:15:16.167  S-1-5-21-996632719-2361334927-4038480536-500  
2 2019-01-15 05:15:16.277  S-1-5-21-996632719-2361334927-4038480536-500  
3 2019-01-15 05:15:16.340  S-1-5-21-996632719-2361334927-4038480536-500  
4 2019-01-15 05:15:16.400  S-1-5-21-996632719-2361334927-4038480536-500

SubjectUserSid   SubjectUserName  \
0    S-1-5-21-996632719-2361334927-4038480536-500        MSTICAdmin   
115                                      S-1-5-18  MSTICAlertsWin1$   

    SubjectDomainName  
0     MSTICAlertsWin1  
115         WORKGROUP

### Simple merge on common key

In [5]:
procs.merge(users, on="SubjectUserSid")

NewProcessId                       NewProcessName  \
0         0x1580       C:\Diagnostics\UserTmp\ftp.exe   
1         0x16fc       C:\Diagnostics\UserTmp\reg.exe   
2         0x1700       C:\Diagnostics\UserTmp\cmd.exe   
3         0x1728  C:\Diagnostics\UserTmp\rundll32.exe   
4         0x175c  C:\Diagnostics\UserTmp\rundll32.exe   
..           ...                                  ...   
112       0x1434  C:\Diagnostics\UserTmp\rundll32.exe   
113       0x123c  C:\Diagnostics\UserTmp\regsvr32.exe   
114        0x240     C:\Windows\System32\tasklist.exe   
115       0x15a0    C:\Windows\System32\win32calc.exe   
116        0xbc8          C:\Windows\System32\cmd.exe   

                                           CommandLine ParentProcessId  \
0                     .\ftp  -s:C:\RECYCLER\xxppyy.exe           0xbc8   
1    .\reg  not /domain:everything that /sid:shines...           0xbc8   
2                   cmd  /c "systeminfo && systeminfo"           0xbc8   
3                             .\rundll32  /C 12345.exe           0xbc8   
4         .\rundll32  /C c:\users\MSTICAdmin\12345.exe           0xbc8   
..                                                 ...             ...   
112    .\rundll32.exe  /C c:\windows\fonts\conhost.exe           0xbc8   
113       .\regsvr32  /u /s c:\windows\fonts\csrss.exe           0xbc8   
114                                           tasklist           0xbc8   
115               "C:\Windows\System32\win32calc.exe"           0x1580   
116  cmd.exe /c c:\Diagnostics\WindowsSimulateDetec...           0x440   

             TimeCreatedUtc                                SubjectUserSid  \
0   2019-01-15 05:15:15.677  S-1-5-21-996632719-2361334927-4038480536-500   
1   2019-01-15 05:15:16.167  S-1-5-21-996632719-2361334927-4038480536-500   
2   2019-01-15 05:15:16.277  S-1-5-21-996632719-2361334927-4038480536-500   
3   2019-01-15 05:15:16.340  S-1-5-21-996632719-2361334927-4038480536-500   
4   2019-01-15 05:15:16.400  S-1-5-21-996632719-2361334927-4038480536-500   
..                      ...                                           ...   
112 2019-01-15 05:15:14.613  S-1-5-21-996632719-2361334927-4038480536-500   
113 2019-01-15 05:15:14.693  S-1-5-21-996632719-2361334927-4038480536-500   
114 2019-01-15 05:15:14.770  S-1-5-21-996632719-2361334927-4038480536-500   
115 2019-01-15 05:15:13.053  S-1-5-21-996632719-2361334927-4038480536-500   
116 2019-01-15 05:15:03.047                                      S-1-5-18   

      SubjectUserName SubjectDomainName  
0          MSTICAdmin   MSTICAlertsWin1  
1          MSTICAdmin   MSTICAlertsWin1  
2          MSTICAdmin   MSTICAlertsWin1  
3          MSTICAdmin   MSTICAlertsWin1  
4          MSTICAdmin   MSTICAlertsWin1  
..                ...               ...  
112        MSTICAdmin   MSTICAlertsWin1  
113        MSTICAdmin   MSTICAlertsWin1  
114        MSTICAdmin   MSTICAlertsWin1  
115        MSTICAdmin   MSTICAlertsWin1  
116  MSTICAlertsWin1$         WORKGROUP  

[117 rows x 8 columns]

### Left joins (also right and outer)

In [6]:
procs.merge(users[1:], on="SubjectUserSid")

NewProcessId               NewProcessName  \
0        0xbc8  C:\Windows\System32\cmd.exe   

                                         CommandLine ParentProcessId  \
0  cmd.exe /c c:\Diagnostics\WindowsSimulateDetec...           0x440   

           TimeCreatedUtc SubjectUserSid   SubjectUserName SubjectDomainName  
0 2019-01-15 05:15:03.047       S-1-5-18  MSTICAlertsWin1$         WORKGROUP

In [7]:
procs.merge(users[1:], on="SubjectUserSid", how="left")

NewProcessId                       NewProcessName  \
0         0x1580       C:\Diagnostics\UserTmp\ftp.exe   
1         0x16fc       C:\Diagnostics\UserTmp\reg.exe   
2         0x1700       C:\Diagnostics\UserTmp\cmd.exe   
3         0x1728  C:\Diagnostics\UserTmp\rundll32.exe   
4         0x175c  C:\Diagnostics\UserTmp\rundll32.exe   
..           ...                                  ...   
112       0x1434  C:\Diagnostics\UserTmp\rundll32.exe   
113       0x123c  C:\Diagnostics\UserTmp\regsvr32.exe   
114        0x240     C:\Windows\System32\tasklist.exe   
115        0xbc8          C:\Windows\System32\cmd.exe   
116       0x15a0    C:\Windows\System32\win32calc.exe   

                                           CommandLine ParentProcessId  \
0                     .\ftp  -s:C:\RECYCLER\xxppyy.exe           0xbc8   
1    .\reg  not /domain:everything that /sid:shines...           0xbc8   
2                   cmd  /c "systeminfo && systeminfo"           0xbc8   
3                             .\rundll32  /C 12345.exe           0xbc8   
4         .\rundll32  /C c:\users\MSTICAdmin\12345.exe           0xbc8   
..                                                 ...             ...   
112    .\rundll32.exe  /C c:\windows\fonts\conhost.exe           0xbc8   
113       .\regsvr32  /u /s c:\windows\fonts\csrss.exe           0xbc8   
114                                           tasklist           0xbc8   
115  cmd.exe /c c:\Diagnostics\WindowsSimulateDetec...           0x440   
116               "C:\Windows\System32\win32calc.exe"           0x1580   

             TimeCreatedUtc                                SubjectUserSid  \
0   2019-01-15 05:15:15.677  S-1-5-21-996632719-2361334927-4038480536-500   
1   2019-01-15 05:15:16.167  S-1-5-21-996632719-2361334927-4038480536-500   
2   2019-01-15 05:15:16.277  S-1-5-21-996632719-2361334927-4038480536-500   
3   2019-01-15 05:15:16.340  S-1-5-21-996632719-2361334927-4038480536-500   
4   2019-01-15 05:15:16.400  S-1-5-21-996632719-2361334927-4038480536-500   
..                      ...                                           ...   
112 2019-01-15 05:15:14.613  S-1-5-21-996632719-2361334927-4038480536-500   
113 2019-01-15 05:15:14.693  S-1-5-21-996632719-2361334927-4038480536-500   
114 2019-01-15 05:15:14.770  S-1-5-21-996632719-2361334927-4038480536-500   
115 2019-01-15 05:15:03.047                                      S-1-5-18   
116 2019-01-15 05:15:13.053  S-1-5-21-996632719-2361334927-4038480536-500   

      SubjectUserName SubjectDomainName  
0                 NaN               NaN  
1                 NaN               NaN  
2                 NaN               NaN  
3                 NaN               NaN  
4                 NaN               NaN  
..                ...               ...  
112               NaN               NaN  
113               NaN               NaN  
114               NaN               NaN  
115  MSTICAlertsWin1$         WORKGROUP  
116               NaN               NaN  

[117 rows x 8 columns]

### Joins where no common key

In [8]:
(
    procs.merge(parents, left_on="ParentProcessId", right_on="ProcessId")
    .head()
    .filter(regex=".*Process.*")
)

NewProcessId                       NewProcessName ParentProcessId ProcessId  \
0       0x1580       C:\Diagnostics\UserTmp\ftp.exe           0xbc8     0xbc8   
1       0x16fc       C:\Diagnostics\UserTmp\reg.exe           0xbc8     0xbc8   
2       0x1700       C:\Diagnostics\UserTmp\cmd.exe           0xbc8     0xbc8   
3       0x1728  C:\Diagnostics\UserTmp\rundll32.exe           0xbc8     0xbc8   
4       0x175c  C:\Diagnostics\UserTmp\rundll32.exe           0xbc8     0xbc8   

             ParentProcessName  
0  C:\Windows\System32\cmd.exe  
1  C:\Windows\System32\cmd.exe  
2  C:\Windows\System32\cmd.exe  
3  C:\Windows\System32\cmd.exe  
4  C:\Windows\System32\cmd.exe

---

# Using Styles [Ian] 

- Max/min values 
- Value coloring 
- Inline bars 

<p style="font-family:consolas; font-size:15pt; color:green">
df.style(...)
</p>

In [9]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

# Generate a summary
summary_df = (
    net_df[["RemoteRegion", "TotalAllowedFlows", "L7Protocol"]]
    .groupby("RemoteRegion")
    .agg(
        FlowsSum = pd.NamedAgg("TotalAllowedFlows", "sum"),
        FlowsVar = pd.NamedAgg("TotalAllowedFlows", "var"),
        FlowsStdDev = pd.NamedAgg("TotalAllowedFlows", "std"),
        L7Prots = pd.NamedAgg("L7Protocol", "nunique"),
    )
)
summary_df

FlowsSum   FlowsVar  FlowsStdDev  L7Prots
RemoteRegion                                             
                   814.0  57.267027     7.567498        6
canadacentral     5103.0  29.811223     5.459965        1
centralus          236.0   4.675897     2.162382        1
eastus             602.0   1.646154     1.283025        3
eastus2           1502.0   4.830914     2.197934        1
northeurope         82.0   0.492438     0.701739        1
southcentralus     817.0   8.882186     2.980300        1
westcentralus       59.0   0.017241     0.131306        1
westus              38.0   0.782609     0.884652        1
westus2              7.0   0.300000     0.547723        1

### highlight_max/highlight_mix

<p style="font-family:consolas; font-size:15pt; color:green">
df.style.highlight_max(...)
</p>

In [10]:
df_style = summary_df.style.highlight_max(color="blue").highlight_min(color="green")
df_style

### Color gradients

<p style="font-family:consolas; font-size:15pt; color:green">
df.style.background_gradient(...)
</p>

In [11]:
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)

summary_df.style.background_gradient(cmap=cm).format("{:.2f}")

### Inline bars

<p style="font-family:consolas; font-size:15pt; color:green">
df.style.bar(...)
</p>


In [12]:
summary_df.style.bar(color="blue").format("{:.2f}")

### Custom properties

<p style="font-family:consolas; font-size:15pt; color:green">
df.style.set_properties(...)
</p>

If you are really into CSS, you can do (almost) anything

In [59]:
summary_df.style.set_properties(**{
    'background-color': 'black',
    'color': 'lawngreen',
    'font-family': 'consolas',
}).format("{:.2f}")


---

# Reshaping/preprocessing data? [Ashwin, Luis Francisco, Ian] 

- Dealing with nulls/NAs 
- Type conversion 
- Renaming columns
- Pandas operations: melt, explode, transpose, indexing/stack/unstack 
- Dealing with complex Python objects - explode 
- Tidy data - melt 


## Dealing with nulls/NAs

[Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#missing-data)

pandas primarily uses `NaN` to represent missing data which is of `floattype`. 
IEEE 754 floating point representation of Not a Number (NaN).

Tip: Often you will see TypeError exceptions about not being able to 
perform an expected operation on a float (when you were expecting the type
to be a string or other object type). This is very likely due NaNs in your data.

Also NaT - is the equivalent of NaN for DateTime data.
 
Sometimes python also raises `None` for missing data. `NoneType` object. 

In [14]:
import pandas as pd
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")
len(net_df)

1360

In [15]:
print(f"Null elements in DataFrame: {net_df.isnull().values.sum()} \n\
Rows with null elements: {net_df.shape[0] - net_df.dropna().shape[0]}")

Null elements in DataFrame: 24 
Rows with null elements: 8


### Which columns have NAs?

<p style="font-family:consolas; font-size:15pt; color:green">
df.isna()<br>
series.isna()
</p>

In [16]:
net_df.isna().any()

TimeGenerated          False
FlowStartTime          False
FlowEndTime            False
FlowIntervalEndTime    False
FlowType               False
ResourceGroup           True
VMName                  True
VMIPAddress            False
PublicIPs               True
SrcIP                  False
DestIP                 False
L4Protocol             False
L7Protocol             False
DestPort               False
FlowDirection          False
AllowedOutFlows        False
AllowedInFlows         False
DeniedInFlows          False
DeniedOutFlows         False
RemoteRegion           False
VMRegion               False
AllExtIPs              False
TotalAllowedFlows      False
dtype: bool

In [17]:
net_df.ResourceGroup.value_counts()

asihuntomsworkspacerg    1352
Name: ResourceGroup, dtype: int64

### Filtering to see which columns have NaNs

You can use `.isna()` on the whole DataFrame or a single column.

In [18]:
net_df[net_df["PublicIPs"].isna()]

TimeGenerated       FlowStartTime         FlowEndTime  \
326 2019-02-13 01:23:47.634 2019-02-13 00:55:10 2019-02-13 00:57:33   
327 2019-02-13 01:23:47.634 2019-02-13 00:55:10 2019-02-13 00:57:33   
336 2019-02-12 22:23:05.399 2019-02-12 21:55:43 2019-02-12 21:55:43   
345 2019-02-12 22:23:05.384 2019-02-12 21:54:04 2019-02-12 21:55:36   
357 2019-02-12 23:23:59.515 2019-02-12 22:22:35 2019-02-12 22:55:37   
413 2019-02-12 18:23:51.853 2019-02-12 17:26:19 2019-02-12 17:44:09   
414 2019-02-12 18:23:51.853 2019-02-12 17:26:19 2019-02-12 17:44:09   
466 2019-02-12 22:23:17.236 2019-02-12 21:55:43 2019-02-12 21:55:43   

    FlowIntervalEndTime   FlowType ResourceGroup VMName VMIPAddress PublicIPs  \
326 2019-02-13 01:00:00  IntraVNet          None   None                  None   
327 2019-02-13 01:00:00  IntraVNet          None   None                  None   
336 2019-02-12 22:00:00  IntraVNet          None   None                  None   
345 2019-02-12 22:00:00  IntraVNet          None   None                  None   
357 2019-02-12 23:00:00  IntraVNet          None   None                  None   
413 2019-02-12 18:00:00  IntraVNet          None   None                  None   
414 2019-02-12 18:00:00  IntraVNet          None   None                  None   
466 2019-02-12 22:00:00  IntraVNet          None   None                  None   

        SrcIP  ... DestPort FlowDirection AllowedOutFlows  AllowedInFlows  \
326  10.0.3.4  ...    445.0             O             3.0             0.0   
327  10.0.3.4  ...    445.0             I             0.0             3.0   
336  10.0.3.5  ...     22.0             O             1.0             0.0   
345  10.0.3.5  ...     22.0             O             6.0             0.0   
357  10.0.3.5  ...     22.0             O            12.0             0.0   
413  10.0.3.4  ...    445.0             O             6.0             0.0   
414  10.0.3.4  ...    445.0             I             0.0             6.0   
466  10.0.3.5  ...     22.0             I             0.0             1.0   

    DeniedInFlows  DeniedOutFlows  RemoteRegion  VMRegion     AllExtIPs  \
326           0.0             0.0                  eastus      10.0.3.5   
327           0.0             0.0                  eastus      10.0.3.4   
336           0.0             0.0                  eastus      10.0.3.4   
345           0.0             0.0                  eastus  104.211.30.1   
357           0.0             0.0                  eastus  104.211.30.1   
413           0.0             0.0                  eastus      10.0.3.5   
414           0.0             0.0                  eastus      10.0.3.4   
466           0.0             0.0                  eastus      10.0.3.5   

    TotalAllowedFlows  
326               3.0  
327               3.0  
336               1.0  
345               6.0  
357              12.0  
413               6.0  
414               6.0  
466               1.0  

[8 rows x 23 columns]

### Removing NaNs with `.dropna`

<p style="font-family:consolas; font-size:15pt; color:red">
df.dropna()        # removes all rows with ANY NaNs<br>
df.dropna(axis=1)     # removes all columns with ANY NaNs
</p>
<p style="font-family:consolas; font-size:15pt; color:green">
df.dropna(how="all")  # removes all rows that are ALL NaNs<br>
df.dropna(axis=1, how="all") # removes all cols that are ALL NaNs<br>
</p>

`dropna()` also supports `inplace=True`. Don't do it!!!


In [19]:
len(net_df.dropna())

1352

### Replacing NaNs with values

<p style="font-family:consolas; font-size:15pt; color:green">
df.fillna(<i>replacement</i>)           # replace NaNs with 'replacement'<br><br>
df[column] = df[column].fillna(<i>replacement</i>)  # replace NaNs in a single column
</p>


In [20]:
net_df2 = net_df.fillna(value="N/A")

In [21]:
net_df2.ResourceGroup.value_counts()

asihuntomsworkspacerg    1352
N/A                         8
Name: ResourceGroup, dtype: int64

## Type Conversion

In [22]:
net_df.dtypes

TimeGenerated          datetime64[ns]
FlowStartTime          datetime64[ns]
FlowEndTime            datetime64[ns]
FlowIntervalEndTime    datetime64[ns]
FlowType                       object
ResourceGroup                  object
VMName                         object
VMIPAddress                    object
PublicIPs                      object
SrcIP                          object
DestIP                         object
L4Protocol                     object
L7Protocol                     object
DestPort                      float64
FlowDirection                  object
AllowedOutFlows               float64
AllowedInFlows                float64
DeniedInFlows                 float64
DeniedOutFlows                float64
RemoteRegion                   object
VMRegion                       object
AllExtIPs                      object
TotalAllowedFlows             float64
dtype: object

<p style="font-family:consolas; font-size:15pt; color:green">
series.column.astype(<i>target</i>)  # convert type<br>
</p>

Target can be a numpy type, a pandas dtype or a friendly string:
- "object"
- "datetime"
- "number"

<p style="font-family:consolas; font-size:15pt; color:green">
df.column.astype(<i>target|{col1: type1, col2, type2...}</i>)  # convert multiple cols<br>
</p>


In [23]:
net_df.TotalAllowedFlows = net_df.TotalAllowedFlows.astype('str')

In [24]:
net_df.TotalAllowedFlows.dtypes

dtype('O')

### Convert using explicit pandas function

Gives you more control over specific conversions (esp for DateTime)

In [25]:
net_df.TotalAllowedFlows = pd.to_numeric(net_df.TotalAllowedFlows)
#pd.to_datetime
#pd.to_timedelta

In [26]:
net_df.TotalAllowedFlows.dtypes

dtype('float64')

## Renaming columns

In [27]:
net_df.columns

Index(['TimeGenerated', 'FlowStartTime', 'FlowEndTime', 'FlowIntervalEndTime',
       'FlowType', 'ResourceGroup', 'VMName', 'VMIPAddress', 'PublicIPs',
       'SrcIP', 'DestIP', 'L4Protocol', 'L7Protocol', 'DestPort',
       'FlowDirection', 'AllowedOutFlows', 'AllowedInFlows', 'DeniedInFlows',
       'DeniedOutFlows', 'RemoteRegion', 'VMRegion', 'AllExtIPs',
       'TotalAllowedFlows'],
      dtype='object')

<p style="font-family:consolas; font-size:15pt; color:green">
df.rename(columns={col1: col1_new, col2: ....})  # rename<br>
</p>


In [28]:
net_df.rename(columns={"FlowStartTime": "FlowStartDateTime", "FlowEndTime": "FlowEndDateTime"}).columns

Index(['TimeGenerated', 'FlowStartDateTime', 'FlowEndDateTime',
       'FlowIntervalEndTime', 'FlowType', 'ResourceGroup', 'VMName',
       'VMIPAddress', 'PublicIPs', 'SrcIP', 'DestIP', 'L4Protocol',
       'L7Protocol', 'DestPort', 'FlowDirection', 'AllowedOutFlows',
       'AllowedInFlows', 'DeniedInFlows', 'DeniedOutFlows', 'RemoteRegion',
       'VMRegion', 'AllExtIPs', 'TotalAllowedFlows'],
      dtype='object')


<p style="font-family:consolas; font-size:15pt; color:green">
df.rename(<i>func</i>, axis='columns')
</p>

In [29]:
net_df.rename(str.lower, axis='columns').columns

Index(['timegenerated', 'flowstarttime', 'flowendtime', 'flowintervalendtime',
       'flowtype', 'resourcegroup', 'vmname', 'vmipaddress', 'publicips',
       'srcip', 'destip', 'l4protocol', 'l7protocol', 'destport',
       'flowdirection', 'allowedoutflows', 'allowedinflows', 'deniedinflows',
       'deniedoutflows', 'remoteregion', 'vmregion', 'allextips',
       'totalallowedflows'],
      dtype='object')

In [30]:
net_df.columns

Index(['TimeGenerated', 'FlowStartTime', 'FlowEndTime', 'FlowIntervalEndTime',
       'FlowType', 'ResourceGroup', 'VMName', 'VMIPAddress', 'PublicIPs',
       'SrcIP', 'DestIP', 'L4Protocol', 'L7Protocol', 'DestPort',
       'FlowDirection', 'AllowedOutFlows', 'AllowedInFlows', 'DeniedInFlows',
       'DeniedOutFlows', 'RemoteRegion', 'VMRegion', 'AllExtIPs',
       'TotalAllowedFlows'],
      dtype='object')

### Statically rename using assignment

In [31]:
net_df.columns = [
    "timegenerated",
    "flowstarttime",
    "flowendtime",
    "flowintervalendtime",
    "flowtype",
    "resourcegroup",
    "vmname",
    "vmipaddress",
    "publicips",
    "srcip",
    "destip",
    "l4protocol",
    "l7protocol",
    "destport",
    "flowdirection",
    "allowedoutflows",
    "allowedinflows",
    "deniedinflows",
    "deniedoutflows",
    "remoteregion",
    "vmregion",
    "allextips",
    "totalallowedflows",
]

## Pandas operations: melt, explode, transpose, indexing/stack/unstack

### Dealing with complex Python objects - explode

In [32]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")
net_df.PublicIPs.head(10)

0                               [13.67.143.117]
1                                [40.77.232.95]
2                  [13.65.107.32, 40.124.45.19]
3                  [13.65.107.32, 40.124.45.19]
4                                [20.38.98.100]
5                               [13.67.143.117]
6                [13.71.172.128, 13.71.172.130]
7                [13.71.172.128, 13.71.172.130]
8    [65.55.44.109, 40.77.228.69, 65.55.44.108]
9    [65.55.44.109, 40.77.228.69, 65.55.44.108]
Name: PublicIPs, dtype: object

In [33]:
net_df.PublicIPs.count()

1352

In [34]:
net_df_ext = net_df.explode("PublicIPs")

In [35]:
net_df_ext.PublicIPs.head(10)

0    13.67.143.117
1     40.77.232.95
2     13.65.107.32
2     40.124.45.19
3     13.65.107.32
3     40.124.45.19
4     20.38.98.100
5    13.67.143.117
6    13.71.172.128
6    13.71.172.130
Name: PublicIPs, dtype: object

In [36]:
len(net_df_ext.PublicIPs.unique())

123

### Tidy data - melt
Pandas.melt() unpivots a DataFrame from wide format to long format.  
melt() function is useful to message a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value. 

In [37]:
net_df_min = net_df[["FlowType", "AllExtIPs", "TotalAllowedFlows"]]

In [38]:
pd.melt(net_df_min, 
         id_vars=['AllExtIPs'])

AllExtIPs           variable        value
0      13.67.143.117           FlowType  AzurePublic
1       40.77.232.95           FlowType  AzurePublic
2       13.65.107.32           FlowType  AzurePublic
3       40.124.45.19           FlowType  AzurePublic
4       20.38.98.100           FlowType  AzurePublic
...              ...                ...          ...
2715   13.71.172.130  TotalAllowedFlows         23.0
2716    40.77.232.95  TotalAllowedFlows          1.0
2717  52.168.138.145  TotalAllowedFlows          4.0
2718    23.215.98.90  TotalAllowedFlows          2.0
2719    72.21.81.240  TotalAllowedFlows          2.0

[2720 rows x 3 columns]

### Traspose

In [39]:
net_df_min.head().T

0             1             2             3  \
FlowType             AzurePublic   AzurePublic   AzurePublic   AzurePublic   
AllExtIPs          13.67.143.117  40.77.232.95  13.65.107.32  40.124.45.19   
TotalAllowedFlows            1.0           1.0           4.0           4.0   

                              4  
FlowType            AzurePublic  
AllExtIPs          20.38.98.100  
TotalAllowedFlows           1.0

### Indexing, Stack and Unstack

In [40]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

In [41]:
net_df_agg = net_df.groupby("AllExtIPs").agg({"TotalAllowedFlows":['mean', 'min', 'max'],
                                              "AllowedOutFlows":['mean', 'min', 'max'],
                                              "AllowedInFlows":['mean', 'min', 'max']})

In [42]:
net_df_agg.head()

TotalAllowedFlows            AllowedOutFlows             \
                           mean  min   max            mean  min   max   
AllExtIPs                                                               
10.0.3.4               3.333333  1.0   6.0        0.333333  0.0   1.0   
10.0.3.5               3.333333  1.0   6.0        3.000000  0.0   6.0   
104.211.30.1           9.000000  6.0  12.0        9.000000  6.0  12.0   
104.40.17.153          1.750000  1.0   2.0        1.750000  1.0   2.0   
104.43.212.12          2.166667  1.0   4.0        2.166667  1.0   4.0   

              AllowedInFlows            
                        mean  min  max  
AllExtIPs                               
10.0.3.4            3.000000  0.0  6.0  
10.0.3.5            0.333333  0.0  1.0  
104.211.30.1        0.000000  0.0  0.0  
104.40.17.153       0.000000  0.0  0.0  
104.43.212.12       0.000000  0.0  0.0

In [43]:
net_df_agg["TotalAllowedFlows"]["mean"]

AllExtIPs
10.0.3.4          3.333333
10.0.3.5          3.333333
104.211.30.1      9.000000
104.40.17.153     1.750000
104.43.212.12     2.166667
                   ...    
90.130.70.73      1.000000
99.84.104.63      7.000000
99.84.106.178    10.000000
99.84.106.27     10.000000
99.84.106.92     10.000000
Name: mean, Length: 125, dtype: float64

In [44]:
idx = pd.IndexSlice
net_df_agg.loc[:,idx[:,'mean']]

TotalAllowedFlows AllowedOutFlows AllowedInFlows
                           mean            mean           mean
AllExtIPs                                                     
10.0.3.4               3.333333        0.333333       3.000000
10.0.3.5               3.333333        3.000000       0.333333
104.211.30.1           9.000000        9.000000       0.000000
104.40.17.153          1.750000        1.750000       0.000000
104.43.212.12          2.166667        2.166667       0.000000
...                         ...             ...            ...
90.130.70.73           1.000000        1.000000       0.000000
99.84.104.63           7.000000        7.000000       0.000000
99.84.106.178         10.000000       10.000000       0.000000
99.84.106.27          10.000000       10.000000       0.000000
99.84.106.92          10.000000       10.000000       0.000000

[125 rows x 3 columns]

In [45]:
net_df_agg_stacked = net_df_agg.stack()

In [46]:
net_df_agg_stacked.head()

TotalAllowedFlows  AllowedOutFlows  AllowedInFlows
AllExtIPs                                                         
10.0.3.4  mean           3.333333         0.333333        3.000000
          min            1.000000         0.000000        0.000000
          max            6.000000         1.000000        6.000000
10.0.3.5  mean           3.333333         3.000000        0.333333
          min            1.000000         0.000000        0.000000

In [47]:
net_df_agg_stacked.loc[("10.0.3.4","mean"),"TotalAllowedFlows"]

3.3333333333333335

In [48]:
net_df_agg_stacked.unstack().head()

TotalAllowedFlows            AllowedOutFlows             \
                           mean  min   max            mean  min   max   
AllExtIPs                                                               
10.0.3.4               3.333333  1.0   6.0        0.333333  0.0   1.0   
10.0.3.5               3.333333  1.0   6.0        3.000000  0.0   6.0   
104.211.30.1           9.000000  6.0  12.0        9.000000  6.0  12.0   
104.40.17.153          1.750000  1.0   2.0        1.750000  1.0   2.0   
104.43.212.12          2.166667  1.0   4.0        2.166667  1.0   4.0   

              AllowedInFlows            
                        mean  min  max  
AllExtIPs                               
10.0.3.4            3.000000  0.0  6.0  
10.0.3.5            0.333333  0.0  1.0  
104.211.30.1        0.000000  0.0  0.0  
104.40.17.153       0.000000  0.0  0.0  
104.43.212.12       0.000000  0.0  0.0

---
# Pivoting/pivot tables [Ashwin]

### [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)

In [49]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")
net_df.head()

TimeGenerated       FlowStartTime         FlowEndTime  \
0 2019-02-14 13:23:59.512 2019-02-14 12:21:58 2019-02-14 12:21:58   
1 2019-02-14 13:23:59.512 2019-02-14 12:29:02 2019-02-14 12:29:02   
2 2019-02-14 03:26:06.765 2019-02-14 02:08:46 2019-02-14 02:48:45   
3 2019-02-14 03:26:06.765 2019-02-14 02:08:46 2019-02-14 02:48:45   
4 2019-02-14 03:26:06.828 2019-02-14 02:30:56 2019-02-14 02:30:56   

  FlowIntervalEndTime     FlowType          ResourceGroup           VMName  \
0 2019-02-14 13:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
1 2019-02-14 13:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
2 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
3 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
4 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   

  VMIPAddress                     PublicIPs SrcIP  ... DestPort FlowDirection  \
0    10.0.3.5               [13.67.143.117]        ...    443.0             O   
1    10.0.3.5                [40.77.232.95]        ...    443.0             O   
2    10.0.3.5  [13.65.107.32, 40.124.45.19]        ...    443.0             O   
3    10.0.3.5  [13.65.107.32, 40.124.45.19]        ...    443.0             O   
4    10.0.3.5                [20.38.98.100]        ...    443.0             O   

  AllowedOutFlows  AllowedInFlows DeniedInFlows  DeniedOutFlows  \
0             1.0             0.0           0.0             0.0   
1             1.0             0.0           0.0             0.0   
2             4.0             0.0           0.0             0.0   
3             4.0             0.0           0.0             0.0   
4             1.0             0.0           0.0             0.0   

     RemoteRegion  VMRegion      AllExtIPs TotalAllowedFlows  
0       centralus    eastus  13.67.143.117               1.0  
1   westcentralus    eastus   40.77.232.95               1.0  
2  southcentralus    eastus   13.65.107.32               4.0  
3  southcentralus    eastus   40.124.45.19               4.0  
4          eastus    eastus   20.38.98.100               1.0  

[5 rows x 23 columns]

In [50]:
# Prepare groupby dataset to perform pivot. Does expect column with unique values
net_df_grouped = net_df.groupby(['FlowIntervalEndTime','FlowType'])['AllExtIPs'].count().reset_index()
net_df_grouped.head()

FlowIntervalEndTime        FlowType  AllExtIPs
0 2019-02-07 13:00:00     AzurePublic          3
1 2019-02-07 14:00:00     AzurePublic          8
2 2019-02-07 14:00:00  ExternalPublic          1
3 2019-02-07 15:00:00     AzurePublic          5
4 2019-02-07 15:00:00  ExternalPublic          3

In [51]:
net_df_grouped.pivot(index="FlowIntervalEndTime", columns="FlowType", values="AllExtIPs")

FlowType             AzurePublic  ExternalPublic  IntraVNet
FlowIntervalEndTime                                        
2019-02-07 13:00:00          3.0             NaN        NaN
2019-02-07 14:00:00          8.0             1.0        NaN
2019-02-07 15:00:00          5.0             3.0        NaN
2019-02-07 16:00:00          7.0             1.0        NaN
2019-02-07 17:00:00          8.0             NaN        NaN
...                          ...             ...        ...
2019-02-14 09:00:00          8.0             1.0        NaN
2019-02-14 10:00:00          8.0             NaN        NaN
2019-02-14 11:00:00          7.0             2.0        NaN
2019-02-14 12:00:00          9.0             NaN        NaN
2019-02-14 13:00:00          2.0             NaN        NaN

[141 rows x 3 columns]

---
# Time manipulation [Ashwin] 

- Timezone conversions
- Resample - Grouping by time 

In [52]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")
net_df.head()

TimeGenerated       FlowStartTime         FlowEndTime  \
0 2019-02-14 13:23:59.512 2019-02-14 12:21:58 2019-02-14 12:21:58   
1 2019-02-14 13:23:59.512 2019-02-14 12:29:02 2019-02-14 12:29:02   
2 2019-02-14 03:26:06.765 2019-02-14 02:08:46 2019-02-14 02:48:45   
3 2019-02-14 03:26:06.765 2019-02-14 02:08:46 2019-02-14 02:48:45   
4 2019-02-14 03:26:06.828 2019-02-14 02:30:56 2019-02-14 02:30:56   

  FlowIntervalEndTime     FlowType          ResourceGroup           VMName  \
0 2019-02-14 13:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
1 2019-02-14 13:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
2 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
3 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
4 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   

  VMIPAddress                     PublicIPs SrcIP  ... DestPort FlowDirection  \
0    10.0.3.5               [13.67.143.117]        ...    443.0             O   
1    10.0.3.5                [40.77.232.95]        ...    443.0             O   
2    10.0.3.5  [13.65.107.32, 40.124.45.19]        ...    443.0             O   
3    10.0.3.5  [13.65.107.32, 40.124.45.19]        ...    443.0             O   
4    10.0.3.5                [20.38.98.100]        ...    443.0             O   

  AllowedOutFlows  AllowedInFlows DeniedInFlows  DeniedOutFlows  \
0             1.0             0.0           0.0             0.0   
1             1.0             0.0           0.0             0.0   
2             4.0             0.0           0.0             0.0   
3             4.0             0.0           0.0             0.0   
4             1.0             0.0           0.0             0.0   

     RemoteRegion  VMRegion      AllExtIPs TotalAllowedFlows  
0       centralus    eastus  13.67.143.117               1.0  
1   westcentralus    eastus   40.77.232.95               1.0  
2  southcentralus    eastus   13.65.107.32               4.0  
3  southcentralus    eastus   40.124.45.19               4.0  
4          eastus    eastus   20.38.98.100               1.0  

[5 rows x 23 columns]

### Timezone considerations 

In [53]:
dti = pd.to_datetime(net_df['TimeGenerated'])
dti_utc = dti.dt.tz_localize("UTC")
print(dti_utc)

0      2019-02-14 13:23:59.512000+00:00
1      2019-02-14 13:23:59.512000+00:00
2      2019-02-14 03:26:06.765000+00:00
3      2019-02-14 03:26:06.765000+00:00
4      2019-02-14 03:26:06.828000+00:00
                     ...               
1355   2019-02-09 03:32:41.967000+00:00
1356   2019-02-09 03:32:51.124000+00:00
1357   2019-02-09 03:32:51.264000+00:00
1358   2019-02-09 03:32:45.608000+00:00
1359   2019-02-09 03:32:45.608000+00:00
Name: TimeGenerated, Length: 1360, dtype: datetime64[ns, UTC]


In [54]:
dti_pst = dti.dt.tz_localize("US/Pacific")
print(dti_pst)

0      2019-02-14 13:23:59.512000-08:00
1      2019-02-14 13:23:59.512000-08:00
2      2019-02-14 03:26:06.765000-08:00
3      2019-02-14 03:26:06.765000-08:00
4      2019-02-14 03:26:06.828000-08:00
                     ...               
1355   2019-02-09 03:32:41.967000-08:00
1356   2019-02-09 03:32:51.124000-08:00
1357   2019-02-09 03:32:51.264000-08:00
1358   2019-02-09 03:32:45.608000-08:00
1359   2019-02-09 03:32:45.608000-08:00
Name: TimeGenerated, Length: 1360, dtype: datetime64[ns, US/Pacific]


### Grouping by time

[Resampling](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling) - `resample()` time-based groupby

In [55]:
net_df.set_index('TimeGenerated').resample('H')['FlowType'].count()

TimeGenerated
2019-02-07 13:00:00    3
2019-02-07 14:00:00    9
2019-02-07 15:00:00    8
2019-02-07 16:00:00    8
2019-02-07 17:00:00    8
                      ..
2019-02-14 09:00:00    9
2019-02-14 10:00:00    8
2019-02-14 11:00:00    9
2019-02-14 12:00:00    9
2019-02-14 13:00:00    2
Freq: H, Name: FlowType, Length: 169, dtype: int64

---
# Other Useful operations  [Ashwin, Luis Francisco, Ian] 

- Chaining multiple operations with "." 
- Including external functions with pipe 
- Apply, assign, others ???? 

### Chaining multiple operations with "."

In [56]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")
net_df.head()

TimeGenerated       FlowStartTime         FlowEndTime  \
0 2019-02-14 13:23:59.512 2019-02-14 12:21:58 2019-02-14 12:21:58   
1 2019-02-14 13:23:59.512 2019-02-14 12:29:02 2019-02-14 12:29:02   
2 2019-02-14 03:26:06.765 2019-02-14 02:08:46 2019-02-14 02:48:45   
3 2019-02-14 03:26:06.765 2019-02-14 02:08:46 2019-02-14 02:48:45   
4 2019-02-14 03:26:06.828 2019-02-14 02:30:56 2019-02-14 02:30:56   

  FlowIntervalEndTime     FlowType          ResourceGroup           VMName  \
0 2019-02-14 13:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
1 2019-02-14 13:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
2 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
3 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   
4 2019-02-14 03:00:00  AzurePublic  asihuntomsworkspacerg  msticalertswin1   

  VMIPAddress                     PublicIPs SrcIP  ... DestPort FlowDirection  \
0    10.0.3.5               [13.67.143.117]        ...    443.0             O   
1    10.0.3.5                [40.77.232.95]        ...    443.0             O   
2    10.0.3.5  [13.65.107.32, 40.124.45.19]        ...    443.0             O   
3    10.0.3.5  [13.65.107.32, 40.124.45.19]        ...    443.0             O   
4    10.0.3.5                [20.38.98.100]        ...    443.0             O   

  AllowedOutFlows  AllowedInFlows DeniedInFlows  DeniedOutFlows  \
0             1.0             0.0           0.0             0.0   
1             1.0             0.0           0.0             0.0   
2             4.0             0.0           0.0             0.0   
3             4.0             0.0           0.0             0.0   
4             1.0             0.0           0.0             0.0   

     RemoteRegion  VMRegion      AllExtIPs TotalAllowedFlows  
0       centralus    eastus  13.67.143.117               1.0  
1   westcentralus    eastus   40.77.232.95               1.0  
2  southcentralus    eastus   13.65.107.32               4.0  
3  southcentralus    eastus   40.124.45.19               4.0  
4          eastus    eastus   20.38.98.100               1.0  

[5 rows x 23 columns]

In [57]:
net_df[["TenantId","AllExtIPs"]].groupby("AllExtIPs").agg("count")

KeyError: "['TenantId'] not in index"

In [ ]:
net_df[["TenantId","AllExtIPs"]].groupby("AllExtIPs").agg("count").sort_values(by="TenantId", ascending=False)

KeyError: "['TenantId'] not in index"

#### Use parentheses to let you stack the functions vertically

In [ ]:
(
    net_df[["TenantId","AllExtIPs"]]
    .groupby("AllExtIPs")
    .agg("count")
    .sort_values(by="TenantId", ascending=False)
)

In [ ]:
(
    net_df[["TenantId", "AllExtIPs"]]
    .groupby("AllExtIPs")
    .agg("count")
    .sort_values(
        by="TenantId", ascending=False
    )
    .head(5)
)

TenantId
AllExtIPs               
65.55.44.109         139
13.71.172.130        136
52.168.138.145       117
40.124.45.19         115
13.71.172.128        114

In [ ]:
(
    net_df[["TenantId","AllExtIPs"]]
    .groupby("AllExtIPs")
    .agg("count")
    .sort_values(by="TenantId", ascending=False)
    .head(5)
    .index
    .to_list()
)

['65.55.44.109',
 '13.71.172.130',
 '52.168.138.145',
 '40.124.45.19',
 '13.71.172.128']

### External functions with pipe

In [ ]:
def fill_missing_values(df):
    for col in df.columns:
        df[col].fillna("N/A", inplace=True)
    return df

def drop_duplicates(df, column_name):
    df = df.drop_duplicates(subset=column_name)
    return df

In [ ]:
net_df.value_counts("AllExtIPs")

AllExtIPs
65.55.44.109      139
13.71.172.130     136
52.168.138.145    117
40.124.45.19      115
13.71.172.128     114
                 ... 
23.45.181.178       1
23.45.181.176       1
23.45.181.160       1
23.45.180.34        1
99.84.106.92        1
Length: 125, dtype: int64

In [ ]:
net_df.isnull().values.sum()

24

In [ ]:
net_df.pipe(fill_missing_values).isnull().values.sum()

0

In [ ]:
len(net_df)

1360

In [ ]:
net_df.pipe(drop_duplicates, "AllExtIPs").count()

TenantId               125
TimeGenerated          125
FlowStartTime          125
FlowEndTime            125
FlowIntervalEndTime    125
FlowType               125
ResourceGroup          122
VMName                 122
VMIPAddress            125
PublicIPs              122
SrcIP                  125
DestIP                 125
L4Protocol             125
L7Protocol             125
DestPort               125
FlowDirection          125
AllowedOutFlows        125
AllowedInFlows         125
DeniedInFlows          125
DeniedOutFlows         125
RemoteRegion           125
VMRegion               125
AllExtIPs              125
TotalAllowedFlows      125
dtype: int64

In [ ]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

In [ ]:
len(net_df)

1360

In [ ]:
net_df_pipe = (net_df.pipe(drop_duplicates, "AllExtIPs").pipe(fill_missing_values))

In [ ]:
net_df_pipe.isnull().values.sum()

0

### External functions with apply

In [ ]:
import pandas as pd
import numpy as np

def adder(element1, element2):
    return element1+element2


df = pd.DataFrame(np.random.randn(10, 3), columns=['Col1', 'Col2', 'Col3'])
print(df)

df1= df.apply(adder, args=([10]))
#df1= df.apply(adder, element2=10)

print(df1)

       Col1      Col2      Col3
0  0.379748 -0.459915 -1.630365
1  0.733204 -0.477975  0.297341
2 -1.187185  0.367660 -1.602276
3  0.500678  0.226610 -0.431865
4 -0.199604  0.131271 -0.173194
5 -0.063522  0.687468  0.009924
6 -1.137589  1.686923 -0.835308
7 -0.120839 -0.893021 -0.423119
8  0.320839  1.275479 -0.092932
9  1.274244 -0.363499 -1.022512
        Col1       Col2       Col3
0  10.379748   9.540085   8.369635
1  10.733204   9.522025  10.297341
2   8.812815  10.367660   8.397724
3  10.500678  10.226610   9.568135
4   9.800396  10.131271   9.826806
5   9.936478  10.687468  10.009924
6   8.862411  11.686923   9.164692
7   9.879161   9.106979   9.576881
8  10.320839  11.275479   9.907068
9  11.274244   9.636501   8.977488


In [ ]:
net_df.head()

TenantId           TimeGenerated  \
0  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 13:23:59.512   
1  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 13:23:59.512   
2  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 03:26:06.765   
3  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 03:26:06.765   
4  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 03:26:06.828   

        FlowStartTime         FlowEndTime FlowIntervalEndTime     FlowType  \
0 2019-02-14 12:21:58 2019-02-14 12:21:58 2019-02-14 13:00:00  AzurePublic   
1 2019-02-14 12:29:02 2019-02-14 12:29:02 2019-02-14 13:00:00  AzurePublic   
2 2019-02-14 02:08:46 2019-02-14 02:48:45 2019-02-14 03:00:00  AzurePublic   
3 2019-02-14 02:08:46 2019-02-14 02:48:45 2019-02-14 03:00:00  AzurePublic   
4 2019-02-14 02:30:56 2019-02-14 02:30:56 2019-02-14 03:00:00  AzurePublic   

           ResourceGroup           VMName VMIPAddress  \
0  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   
1  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   
2  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   
3  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   
4  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   

                      PublicIPs SrcIP DestIP L4Protocol L7Protocol  DestPort  \
0               [13.67.143.117]                       T      https     443.0   
1                [40.77.232.95]                       T      https     443.0   
2  [13.65.107.32, 40.124.45.19]                       T      https     443.0   
3  [13.65.107.32, 40.124.45.19]                       T      https     443.0   
4                [20.38.98.100]                       T      https     443.0   

  FlowDirection  AllowedOutFlows  AllowedInFlows  DeniedInFlows  \
0             O              1.0             0.0            0.0   
1             O              1.0             0.0            0.0   
2             O              4.0             0.0            0.0   
3             O              4.0             0.0            0.0   
4             O              1.0             0.0            0.0   

   DeniedOutFlows    RemoteRegion VMRegion      AllExtIPs  TotalAllowedFlows  
0             0.0       centralus   eastus  13.67.143.117                1.0  
1             0.0   westcentralus   eastus   40.77.232.95                1.0  
2             0.0  southcentralus   eastus   13.65.107.32                4.0  
3             0.0  southcentralus   eastus   40.124.45.19                4.0  
4             0.0          eastus   eastus   20.38.98.100                1.0

In [ ]:
net_df_ext = net_df.explode("PublicIPs")

In [ ]:
def get_country(ip):
    try:
        x = reader.country(ip)
    except:
        return np.nan
    try:
        return x.country.name if x else np.nan
    except KeyError:
        return np.nan

In [ ]:
!pip install geoip2

In [ ]:
import geoip2.database
with geoip2.database.Reader(r'./Data/GeoLite2-Country.mmdb') as reader:
    unique_ips = net_df_ext['PublicIPs'].unique()
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    net_df_ext['PublicIPCountry'] = net_df_ext['PublicIPs'].map(unique_ips.apply(get_country))

In [ ]:
net_df_ext.value_counts("PublicIPCountry")

PublicIPCountry
United States     1885
Canada             515
Ireland             66
United Kingdom       4
Sweden               2
dtype: int64

In [ ]:
net_df_ext.head()

TenantId           TimeGenerated  \
0  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 13:23:59.512   
1  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 13:23:59.512   
2  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 03:26:06.765   
2  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 03:26:06.765   
3  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 03:26:06.765   

        FlowStartTime         FlowEndTime FlowIntervalEndTime     FlowType  \
0 2019-02-14 12:21:58 2019-02-14 12:21:58 2019-02-14 13:00:00  AzurePublic   
1 2019-02-14 12:29:02 2019-02-14 12:29:02 2019-02-14 13:00:00  AzurePublic   
2 2019-02-14 02:08:46 2019-02-14 02:48:45 2019-02-14 03:00:00  AzurePublic   
2 2019-02-14 02:08:46 2019-02-14 02:48:45 2019-02-14 03:00:00  AzurePublic   
3 2019-02-14 02:08:46 2019-02-14 02:48:45 2019-02-14 03:00:00  AzurePublic   

           ResourceGroup           VMName VMIPAddress      PublicIPs SrcIP  \
0  asihuntomsworkspacerg  msticalertswin1    10.0.3.5  13.67.143.117         
1  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   40.77.232.95         
2  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   13.65.107.32         
2  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   40.124.45.19         
3  asihuntomsworkspacerg  msticalertswin1    10.0.3.5   13.65.107.32         

  DestIP L4Protocol L7Protocol  DestPort FlowDirection  AllowedOutFlows  \
0                 T      https     443.0             O              1.0   
1                 T      https     443.0             O              1.0   
2                 T      https     443.0             O              4.0   
2                 T      https     443.0             O              4.0   
3                 T      https     443.0             O              4.0   

   AllowedInFlows  DeniedInFlows  DeniedOutFlows    RemoteRegion VMRegion  \
0             0.0            0.0             0.0       centralus   eastus   
1             0.0            0.0             0.0   westcentralus   eastus   
2             0.0            0.0             0.0  southcentralus   eastus   
2             0.0            0.0             0.0  southcentralus   eastus   
3             0.0            0.0             0.0  southcentralus   eastus   

       AllExtIPs  TotalAllowedFlows PublicIPCountry  
0  13.67.143.117                1.0   United States  
1   40.77.232.95                1.0   United States  
2   13.65.107.32                4.0   United States  
2   13.65.107.32                4.0   United States  
3   40.124.45.19                4.0   United States

# End of Session
--------------------------------------

# <font color=peru>Break: 5 Minutes</font>

![](../media/dog-leash-break.jpg)